## Introduction to Human In The Loop in Agent Framework

Workflows in the Microsoft Agent Framework allow developers to design, monitor, and control how multiple AI agents and logic components interact to complete complex tasks. They bring structure, flexibility, and transparency to agent-driven applications.

One of the ways to bring in control and manage agent workflows is to add human approvals. When agents require any user input, for example to approve a function call, this is referred to as a **human-in-the-loop** pattern. 

An agent run that requires user input will complete with a response that indicates what input is required from the user instead of completing with a final answer. The caller of the agent is then responsible for getting the required input from the user and passing it back to the agent as part of a new agent run.


In [2]:
import os
from dotenv import load_dotenv
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

load_dotenv()

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
deployment_name = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_AI_API_VERSION")
credential = AzureCliCredential()


chat_client = AzureOpenAIChatClient(
    endpoint=project_endpoint,
    ad_token=credential.get_token("https://cognitiveservices.azure.com/.default").token,
    deployment_name=deployment_name,
    api_version=api_version
)


### Example 1 - Function Tools with Human-In-The-Loop approvals



#### Key Steps

1. `@ai_function(approval_mode="always_require")`:

Use the `@ai_function` decorator to specify metadata. To create a function that requires approval, you can use the `approval_mode` parameter so the agent can pause and ask for user approval.

2. First agent Run and User Input Request:

This agent run `result` is an `AgentRunResponse` object which has a `user_input_requests` attribute of type `list[UserInputRequestContents]`. The run returns one `UserInputRequestContent` type message per tool call, and often there are several approval requests per agent run so it's important to inspect all pending approvals:

```python
    if result.user_input_requests:
        print("Approval required:")
        for user_input_needed in result.user_input_requests:
            print(f"Function: {user_input_needed.function_call.name}")

```


Since you now have a function that requires approval, the agent can respond with a request for approval instead of executing the function directly and returning the result.You should check the agent run response for any user input requests.

The details of the function call including name and arguments can be found in the `function_call` property on the user input request:

```python
    user_input_needed.function_call.name
    user_input_needed.function_call.arguments
```

3. Create the Approval Response:

Once the user has provided their input, you can create a response using the `.create_response()` method on the user input request. Pass True to approve the function call, or False to reject it.

The response should then be passed to the agent in a new ChatMessage:

```python
    approval_message = ChatMessage(
    role=Role.USER, 
        contents=[user_input_needed.create_response(user_approval)]
    )
```
`.create_response(user_approval)` links the approval response back to the original function call request (`user_input_needed`) and embeds the approval decision (boolean) inside a `FunctionApprovalResponseContent` type message the framework understands.

4. **Resuming Execution:**

After collecting user approval decisions for pending function calls, you need to resume the agent's execution by reconstructing the conversation history. To resume execution, you must provide the original query, the agent's function call requests and the user's approval responses as ChatMessage objects:
    
```python
    final_result = await agent1.run([
    "What is the detailed weather like in Amsterdam?",
    ChatMessage(role=Role.ASSISTANT, contents=[user_input_needed]),
    approval_message
    ])
```
By providing both messages, you're reconstructing the conversation flow: the agent proposed an action (ASSISTANT), and the user responded to that proposal (USER).

> *Note*: In this example, if the user rejects a function call (`user_approval = False`), the agent doesn't simply fail. Instead, it attempts to fulfill the user's intent using alternative available tool - `get_weather`. In this example, if `get_weather_detail` is rejected, the agent may fall back to calling `get_weather` (which doesn't require approval) to still provide some weather information.

In [3]:
from typing import Annotated
from agent_framework import ai_function, ChatMessage, Role, ChatAgent 


@ai_function
def get_weather(location: Annotated[str, "The city and state, e.g. San Francisco, CA"]) -> str:
    """Get the current weather for a given location."""
    return f"The weather in {location} is sunny with a high of 30°C."

@ai_function(approval_mode="always_require")
def get_weather_detail(location: Annotated[str, "The city and state, e.g. San Francisco, CA"]) -> str:
    """Get detailed weather information for a given location."""
    return f"The weather in {location} is cloudy with a high of 15°C, humidity 88%."


agent = ChatAgent(
    chat_client=chat_client,
    name="WeatherAgent",
    instructions="You are a helpful weather assistant.",
    tools=[get_weather_detail, get_weather],
)

result = await agent.run("What is the detailed weather like in Amsterdam?")

if result.user_input_requests:
    print("Approval required:")
    for user_input_needed in result.user_input_requests:
        print(f"Function: {user_input_needed.function_call.name}")
        print(f"Arguments: {user_input_needed.function_call.arguments}")
        
    # Get user approval (non-interactive here)
    user_approval = True 

    # Create the approval response
    approval_message = ChatMessage(
        role=Role.USER, 
        contents=[user_input_needed.create_response(user_approval)]
    )

    final_result = await agent.run([
        "What is the detailed weather like in Amsterdam?",
        ChatMessage(role=Role.ASSISTANT, contents=[user_input_needed]),
        approval_message
    ])

print(final_result.text)

Approval required:
Function: get_weather_detail
Arguments: {"location":"Amsterdam"}
The detailed weather in Amsterdam is currently cloudy, with a high temperature of 15°C and humidity at 88%.


In the example above, during the first agent run, the agent doesn't complete execution because it identifies that `get_weather_detail`, which requires approval, needs to be called. Instead of executing the function, the first `agent.run()` call completes and returns the pending function call in `result.user_input_requests`.

We then collect the user's approval decision and continue execution via the second `agent.run()` call with the complete conversation history, including the approval response. 

> *Note*: When we don't have a thread, we need to pass the full conversation history in the second agent run - the query, all requests and their associated responses, so we're not actually 'resuming' the first run but starting from the beginning (but now we can proceed with tool execution since we have collected the approvals)

This pattern ensures human oversight while giving your application full control over how approval requests are presented to users.

### Example 2 - Handling Approvals in a Loop

The following example registers two tools (`get_user` and `submit_user_data`) and maintains a running conversation throughout the session. When the agent needs to call functions requiring approval, it returns all approval requests. We then collect user input for each pending function call and send all approval responses back in a single follow-up agent run.

The framework automatically:
- Retrieves the full conversation context from the thread
- Matches approvals to pending function calls by `call_id`
- Executes approved functions and continues the conversation

This means you only send new information (the approvals) on each run while the framework handles state management behind the scenes.

First, let's define some simple tools. In reality, these would typically interact with actual databases and external APIs:

In [4]:
from agent_framework import ai_function
from typing import Annotated


@ai_function(approval_mode="always_require")
def submit_user_data(
    name: Annotated[str, "User's full name"],
    role: Annotated[str, "User's role or department"],
    user_id: Annotated[int, "User's ID number"],
) -> str:
    """
    Function that submits user data to the system. 
    This function requires approval.
    When the user asks to submit data, call this function.
    """
    
    return (
        f"User data for '{name}' (ID: {user_id}, role: {role}) has been submitted "
        f"to the system."
    )


@ai_function(
    description="Retrieves the current user's information"
)
def get_user() -> dict:
    """
    Returns a dict with user information including name, role, and ID.
    This function does not require approval.
    """
    # Static user data return
    return {
        "name": "John Doe",
        "role": "Engineering Manager",
        "user_id": 1042
    }

With our tools defined, we can now implement the main interaction loop that handles user inputs:

In the first agent run, we pass the user's query along with the thread that persists only for the duration of the console session:
```python
    thread = agent.get_new_thread()
    result = await agent.run(user_input, thread=thread)
```

Threads maintain conversation history automatically, eliminating the need for manual context reconstruction. The framework treats each subsequent run as a continuation of the same conversation and uses the `call_id` to link each approval response to its corresponding function call request.

In the second agent run, we only need to pass the user approval message and the same thread as in the first run so every turn references the same context:

```python
        # collect all approvals
        approval_messages.append(
                    ChatMessage(role=Role.USER, contents=[request.create_response(approved)])
                )
    followup = await agent.run(approval_messages, thread=thread)    
```

In [5]:
from agent_framework import ChatAgent, ChatMessage, Role
from agent_framework import (
    TextContent,
    FunctionCallContent,
    FunctionResultContent
)

# Initialize the agent with tools and instructions
agent = ChatAgent(
    chat_client=chat_client,
    name="UserAgent",
    instructions=(
        "You are an agent for user management operations." 
        "You assist with submitting and retrieving current user information."
        "For submitting user data, you need: name, role, and user ID."
    ),
    tools=[get_user, submit_user_data],
)

thread = agent.get_new_thread()

print("User Management Agent - Handling Approvals in a Loop")
print("Type 'quit' or 'q' to exit.\n")

while True:

    # Get user input
    user_input = input("You: ").strip()
    if user_input.lower() in ("quit", "q"):
        print(" Finished the session")
        break
    if not user_input:
        continue

    print("\nAgent: ", end="", flush=True)

    # First agent run: process user query
    run1 = await agent.run(user_input, thread=thread)

    print("TYPE RESULT:", type(result))
    print("RESPONSE ID: ", result.response_id)
    print("RESULT INFO: ")
    for msg in run1.messages:
        print("RESULT HAS THIS: ", msg)

    # Check if the agent needs approval for any function calls
    if run1.user_input_requests:
        print("\n\nAPPROVALS REQUIRED:")
        approval_messages: list[ChatMessage] = []

        # Iterate through each approval request and collect user decisions
        for request in run1.user_input_requests:
            print(f"- Approval needed for: {request.function_call.name}")
            print(f"  Arguments: {request.function_call.arguments}")
            print(f"  Function call ID: {request.function_call.call_id}")

            # Get user's approval decision
            approved = input(f"Approve '{request.function_call.name}'? (yes/no): ").strip().lower() == "yes"

            # Add the user's approval response 
            approval_messages.append(
                ChatMessage(role=Role.USER, contents=[request.create_response(approved)])
            )

        # Second agent run: submit approvals and continue execution
        followup = await agent.run(approval_messages, thread=thread)

        print("\nAgent:", followup.text)
        print("\nUser:", run1.text)

    else:
        # No approvals needed - display the response directly
        print(run1.text)
    
    print()


print("RESULT:",run1.messages)
for msg in run1.messages:
    print(f" - {msg.role}: {msg.text}")

User Management Agent - Handling Approvals in a Loop
Type 'quit' or 'q' to exit.


Agent: TYPE RESULT: <class 'agent_framework._types.AgentRunResponse'>
RESPONSE ID:  chatcmpl-Cxxo5dgCfbELc4WwkkLxXVJSUZ0JP
RESULT INFO: 
RESULT HAS THIS:  <agent_framework._types.ChatMessage object at 0x000001F6A93FD850>
RESULT HAS THIS:  <agent_framework._types.ChatMessage object at 0x000001F6A8DEB680>
RESULT HAS THIS:  <agent_framework._types.ChatMessage object at 0x000001F6A94C86B0>
Your current user data is as follows:

- **Name**: John Doe
- **Role**: Engineering Manager
- **User ID**: 1042

 Finished the session
RESULT: [<agent_framework._types.ChatMessage object at 0x000001F6A93FD850>, <agent_framework._types.ChatMessage object at 0x000001F6A8DEB680>, <agent_framework._types.ChatMessage object at 0x000001F6A94C86B0>]
 - assistant: 
 - tool: 
 - assistant: Your current user data is as follows:

- **Name**: John Doe
- **Role**: Engineering Manager
- **User ID**: 1042


##### Recap - ChatMessage Content Types
Agents communicate with applications via different message content types (`TextContent`, `FunctionCallContent`, `FunctionApprovalRequestContent`, etc.). Messages are represented by the `ChatMessage` class and all content type classes inherit from the base `BaseContent` class.

Below is a short reference for some types to help you identify what you can extract from each:

| Content Type              | Description                                                    | Key Properties                                           |
|---------------------------|----------------------------------------------------------------|----------------------------------------------------------|
| TextContent               | Textual content that can be both input and output from the agent | `text`                                                   |
| FunctionCallContent       | A request by an AI service to invoke a function tool          | `name`, `arguments`, `call_id`, `exception`                           |
| FunctionResultContent     | The result returned from a function tool invocation             | `result`, `call_id`, `exception`                                        |
| ErrorContent              | Error information when processing fails                    | `message`, `error_code`, `details`                             |
| UsageContent              | Token usage and billing information from the AI service.               | `details` |

For a detailed description of all available content types and their use, refer to the [`agent_framework` library docs](https://learn.microsoft.com/en-us/python/api/agent-framework-core/agent_framework?view=agent-framework-python-latest) and [AF docs](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/running-agents?pivots=programming-language-python)


Let's use the conversation history stored in the thread to understand what happened during the conversation and what information is retrievable from the messages:

In [6]:
messages = await thread.message_store.list_messages()

print("\n=== FULL THREAD HISTORY ===")
for i, msg in enumerate(messages):
    print(f"\n[Message {i+1}] Role: {msg.role}")
    
    for j, content in enumerate(msg.contents):
        if isinstance(content, TextContent):
            print(f"   Text Content:")
            print("      ", content.text)
        
        elif isinstance(content, FunctionCallContent):
            print(f"   Function Call Content:")
            print(f"      Name: {content.name}")
            print(f"      Arguments: {content.arguments}")

        elif isinstance(content, FunctionResultContent): 
            print(f"   Function Result Content:")
            print("      ", content.result)
        
        else:
            print(f"   Unknown content type: {type(content)}")


=== FULL THREAD HISTORY ===

[Message 1] Role: user
   Text Content:
       get my data

[Message 2] Role: assistant
   Function Call Content:
      Name: get_user
      Arguments: {}

[Message 3] Role: tool
   Function Result Content:
       {'name': 'John Doe', 'role': 'Engineering Manager', 'user_id': 1042}

[Message 4] Role: assistant
   Text Content:
       Your current user data is as follows:

- **Name**: John Doe
- **Role**: Engineering Manager
- **User ID**: 1042


More Samples:
- [Handling function approvals without threads](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/tools/ai_function_with_approval.py)
- [Workflows with function approvals](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/workflows/human-in-the-loop/agents_with_approval_requests.py)  

### Request and Response


In AF, agent execution is fundamentally about the **data flow** - a workflow will keep running as long as there is data being passed. Agents don't "call" each other or pass control in a traditional procedural sense. Instead, they communicate through typed messages flowing between executors through edges.

**Executors** are where computation happens. When an executor emits a message, it flows along the edges to downstream executors which activate when their dependent data becomes available.


#### User Approval Content

The content of the message can also be used to let the application approve or disapprove some actions the agent wants to take.
**User Approval Content** type enables human-in-the-loop approvals within the standard message flow.


The approval mechanism intercepts the normal tool invocation flow, pausing execution until human approval is granted or denied:

<img src="images/reqresp.png" alt="User Approval Content Type" style="width:70%; height:40%; margin:20px auto">

When a ChatAgent needs to invoke a Tool that requires approval:

1. **Initial Inference**: The ChatAgent uses the Model to make an inference and determines it needs to invoke the Tool.

2. **Approval Request**: Before executing, the Tool sends a request back to the ChatAgent, which surfaces to the Application as `FunctionApprovalRequestContent` (a special ChatMessage content type).

3. **User Response**: The Application presents this approval request to the user, who responds (Yes/No). This response is sent back to the ChatAgent as `FunctionApprovalResponseContent`.

4. **Tool Execution**: Once the ChatAgent receives approval, the Tool executes and returns its result to the ChatAgent.

5. **Final Inference**: The Model makes another inference using the Tool's result to generate a natural language response as `TextContent`.

6. **Complete Response**: The Application receives both the `FunctionResultContent` (the raw tool output) and the `TextContent` (the Model's interpretation of that result).

#### User Approval Importance

User approvals are necessary in many cases, especially when tool execution involves external resources or has side effects. Common scenarios include:

- **Irreversible operations**: Data deletion, financial transactions, resource provisioning
- **External system access**: Database modifications, API calls, file system operations
- **Security-sensitive actions**: Credential access, permission changes, privileged operations

When a tool requires approval, the agent pauses execution and transfers control to the application. The agent preserves its state (conversation history, pending actions, context) and waits for the application's decision before proceeding. This mechanism ensures critical operations require explicit human authorization while maintaining the agent's execution context for immediate resumption.

As covered in Chapter 1, **Agent Middleware** provides a general-purpose mechanism for intercepting agent interactions at three levels: agent (intercepts entire agent execution), function (intercepts individual tool invocations), and chat (intercepts LLM calls). You can think of the process of user approval during tool invocation as *tool middleware*. 

TODO1 - requests and responses:
https://learn.microsoft.com/en-us/agent-framework/tutorials/workflows/requests-and-responses?pivots=programming-language-python


TODO2 - https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/workflows/agents/workflow_as_agent_human_in_the_loop.py

TODO3 - https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/workflows/human-in-the-loop/guessing_game_with_human_input.py
